# Creating Intervals, second method

In the first methode we based the creation of the interval on the global review number. If an user have 21 review in the the dataset we will be in the interval 20-29. But in this second method we only care about the number of review in the trainig dataset, if an user have 21 ratings he will probably have 17 review in the training dataset and 4 in the testing dataset then he should go in the interval 10-19 and not 20-29.

To be sure of the separation we did it user by user, because for users who don't have a lot of review, especialy between 1 and 10, a global separtation will pose problem. An User who have 2 review can have the both review in the training or the testing dataset and we expecte one in th trainig and one inthe testing in that case. Then we do the separation differently for each number of review

The second problem is that I want to do some cross validation to improve the results and that will make things more difficult, beacause evry part of the dataset should be test two time, we want 80% in the training dataset and 20% in the testing dataset and we will do 10 folds of cross validation.

For the user who have a number of rating between 1 and 10, we will take only one ratings for the testing dataset

In [1]:
import pandas as pd
import numpy as np
import os.path
import os
from random import *

In [2]:
df=pd.read_parquet('data/ratings.gzip')
df.head()

,userId,movieId,rating
0,48587.0,1.0,3.0
1,48587.0,4.0,2.0
2,48587.0,22.0,3.0
3,48587.0,24.0,3.0
4,48587.0,25.0,3.0


In [3]:
ar=df.to_numpy()

In [4]:
len(df)

6858421

In [5]:
def inteName2(i):
    if i==0:
        return "1-9"
    elif i<10:
        return str(int(i*10))+"-"+str(int(i*10+9))
    elif i>=50:
        return 'sup500'
    else :
        return str(int(i*10))+"-"+str(int(i*10+49))

In [6]:
def inteName(i):
    if i==0:
        return "1-9"
    else :
        return str(int(i*10))+"-"+str(int(i*10+9))

This function return the last line such as the userId is the same as the line l

In [7]:
def nextUser(ar,l):
    userID=ar[l,0]
    t=len(ar)
    nxt=l+1
    while((nxt<t)and(ar[nxt,0]==userID)):
        nxt+=1
    return nxt-1

This function return the list of review which will be in each testing dataset for each fold of the cross validation

In [8]:
def listInteTestInf10(l):
    listInte=list()
    size=len(l)
    cpt=0
    
    if size ==10:
        for i in range(size):
            listInte.append(l[i])
            shuffle(listInte)
        return listInte
    
    while (cpt+size<10) :
        for i in range(size):
            listInte.append(l[i])
            cpt+=1
    if cpt!=10:
        for i in range(10-cpt):
            listInte.append(l[randint(0, size-1)])
    shuffle(listInte)
    return listInte

This function return the list of list of review which will be in each testing dataset for each fold of the cross validation

In [9]:
def listInteTestSup10(l):
    listInte=list()
    size=len(l)
    nbLine=size//5
    for turn in range(2):
        l2=l.copy()
        shuffle(l2)
        for i in range(4):
            
            lTemp=list()
            for k in range(nbLine):
                lTemp.append(l2[0])
                del l2[0]
            listInte.append(lTemp)
        listInte.append(l2)
    return listInte

this fuction add all the review of the user who have all there ratings between start and end. Each line will be add in the good testing dataset and the good trainig dataset for each fold of the cross validation

In [10]:
def listAppend(start,end):
    sizeInte=end-start+1
    inte=list()
    for i in range(start,end+1):
        inte.append(i)
    
    
    if sizeInte<=10 :
        lTest=listInteTestInf10(inte)
        for fold in range(10) :
            dicoNb[fold][ar[start,0]]=(sizeInte-1,1)
            t=lTest[fold]
            for l in inte:
                if l==t :
                    dicoTest[fold].append(l)
                else :
                    dicoTrain[fold].append(l)
    
    else :
        lTest=listInteTestSup10(inte)
        for fold in range(10) :
            t=lTest[fold]
            dicoNb[fold][ar[start,0]]=(sizeInte-len(t),len(t))
            for l in inte :
                if l in t :
                    dicoTest[fold].append(l)
                else :
                    dicoTrain[fold].append(l)

This part of the code create all the list of array which will be use to create all the dataset, we put them in a dictionary. We also record the number of review for each user. All user who have only one review are put in a diffrent dataset

In [11]:
lastLine=len(df)-1
start=0
end=nextUser(ar,start)
dicoTrain={}
dicoTest={}
dicoNb={}
listOne=list()
for fold in range(10):
    dicoNb[fold]={}
    dicoTrain[fold]=list()
    dicoTest[fold]=list()
    
while (end!=lastLine):
    if end!=start:
        listAppend(start,end)
    else :
        listOne.append([ar[start]])
    start=end+1
    end=nextUser(ar,start)
if end!=start:
    listAppend(start,end)

We save the dataset whith all the useer who have only one ratings

In [12]:
arOne=np.concatenate(listOne)
dfOne=pd.DataFrame({'userId':arOne[:,0],'movieId':arOne[:,1],'rating':arOne[:,2]})
dfOne.to_parquet('dataCV3/dfOne.gzip', compression='gzip',index=False)
print(len(dfOne))
dfOne.head()

1402


,userId,movieId,rating
0,124607.0,173145.0,4.5
1,186665.0,455.0,3.5
2,49512.0,174055.0,5.0
3,262971.0,3263.0,4.0
4,204024.0,318.0,4.0


In [13]:
del listOne,arOne,dfOne

We save the number of review for each user for each fold

In [14]:
for fold in range(10):
    print("--== save nb ratings ==-- ")
    ID=list(dicoNb[fold].keys())
    nbTrain=list()
    nbTest=list()
    for i in ID:
        (a,b)=dicoNb[fold][i]
        nbTrain.append(a)
        nbTest.append(b)
    dfDico = pd.DataFrame({'userId' : ID, 'NBRatingTrain' : nbTrain,'NBRatingTest' : nbTest})
    dfDico.to_csv('dataCV3/user-nbRating_fold_'+str(fold)+'.csv',index =False)
    del ID,dfDico
del dicoNb

--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 


We create all the dataset and we save them

In [15]:
for fold in range(10):
    print("--== ",fold," ==-- ")
    print("create and save testing dataset")
    ldf=list()
    for l in dicoTest[fold]:
        ldf.append([ar[l]])
    arTest=np.concatenate(ldf)
    dfTest=pd.DataFrame({'userId':arTest[:,0],'movieId':arTest[:,1],'rating':arTest[:,2]})
    dfTest.to_parquet('dataCV3/dfTest_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del arTest,dfTest
    
    print("create and save training dataset")
    ldf=list()
    for l in dicoTrain[fold]:
        ldf.append([ar[l]])
    arTrain=np.concatenate(ldf)
    dfTrain=pd.DataFrame({'userId':arTrain[:,0],'movieId':arTrain[:,1],'rating':arTrain[:,2]})
    dfTrain.to_parquet('dataCV3/dfTrain_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del arTrain,dfTrain
del ldf

--==  0  ==-- 
create and save testing dataset
create and save training dataset
--==  1  ==-- 
create and save testing dataset
create and save training dataset
--==  2  ==-- 
create and save testing dataset
create and save training dataset
--==  3  ==-- 
create and save testing dataset
create and save training dataset
--==  4  ==-- 
create and save testing dataset
create and save training dataset
--==  5  ==-- 
create and save testing dataset
create and save training dataset
--==  6  ==-- 
create and save testing dataset
create and save training dataset
--==  7  ==-- 
create and save testing dataset
create and save training dataset
--==  8  ==-- 
create and save testing dataset
create and save training dataset
--==  9  ==-- 
create and save testing dataset
create and save training dataset


In [16]:
del dicoTrain,dicoTest

We create all the interval dataset

In [23]:
for fold in range(10):
    print("______________________________________________________________")
    print("fold : ",fold)
    print("load nb ratings")
    arDico=pd.read_csv('dataCV3/user-nbRating_fold_'+str(fold)+'.csv').to_numpy()
    d={}
    for i in range(len(arDico)) :
        d[arDico[i,0]]=arDico[i,1]
    del arDico
    
    print("load testing dataset")
    arTest=pd.read_parquet('dataCV3/dfTest_fold_'+str(fold)+'.gzip').to_numpy()
    size=len(arTest)
    dicoDf={}
    print("Repartition Start")
    for j in range(size): 
        nb=d[arTest[j,0]]//10
        if nb in dicoDf :
            dicoDf[nb].append([arTest[j]])
        else:
            dicoDf[nb]=list()
            dicoDf[nb].append([arTest[j]])

    print("Sauvegarde des intervalles")
    for j in dicoDf:
        a=np.concatenate(dicoDf[j],axis=0)
        dfTemp = pd.DataFrame({'userId':a[:,0],'movieId':a[:,1],'rating':a[:,2]})
        dfTemp.to_parquet('dataCV3/df'+inteName(j)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
        del dfTemp,a
    
    print("Creation of the large intervalles")
    for i in range(10,50,5):
        listDF=list()
        for j in range(5):
            listDF.append(pd.read_parquet('dataCV3/df'+inteName(i+j)+'_fold_'+str(fold)+'.gzip'))
        dfTemp=pd.concat(listDF)
        dfTemp.to_parquet('dataCV3/df'+inteName2(i)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    
    print("Creation of the intervalle sup500")
    listDF=list()
    for i in range(50,800):
        path='dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip'
        if os.path.exists(path):
            listDF.append(pd.read_parquet(path))
    dfTemp=pd.concat(listDF)
    dfTemp.to_parquet('dataCV3/df'+inteName2(50)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del dfTemp,listDF
    
    print("Creation of the intervalle inf20")
    df1=pd.read_parquet('dataCV3/df1-9'+'_fold_'+str(fold)+'.gzip')
    df2=pd.read_parquet('dataCV3/df10-19'+'_fold_'+str(fold)+'.gzip')
    dfInf20=pd.concat([df1,df2])
    dfInf20.to_parquet('dataCV3/dfInf20'+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del df1,df2,dfInf20
    
    print("Creation of the intervalle sup20")
    listDF=list()
    for i in range(2,10):
        listDF.append(pd.read_parquet('dataCV3/df'+inteName2(i)+'_fold_'+str(fold)+'.gzip'))
    for i in range(10,51,5):
        listDF.append(pd.read_parquet('dataCV3/df'+inteName2(i)+'_fold_'+str(fold)+'.gzip'))
    dfSup20=pd.concat(listDF)
    dfSup20.to_parquet('dataCV3/dfSup20_fold_'+str(fold)+'.gzip', compression='gzip',index=False)    

______________________________________________________________
fold :  0
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  1
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  2
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  3
load nb ratings
load testing dataset
Repartition Start
Sauve

In [24]:
for fold in range(10):
    print('- ',fold,' -')
    for i in range(10,800):
        path='dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip'
        if os.path.exists(path):
            os.remove(path)

-  0  -
-  1  -
-  2  -
-  3  -
-  4  -
-  5  -
-  6  -
-  7  -
-  8  -
-  9  -
